In [1]:
import os
import librosa
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
gtzan_dir = 'GTZAN/genres_original/'

In [3]:
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
          'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}


In [6]:
def get_features(y, sr):
    # Features to concatenate in the final dictionary  
    features = {'centroid': None, 'rolloff': None, 'contrast': None, 'bandwidth': None, 'flatness': None,
                'flux': None, 'rmse': None, 'zcr': None, 'chroma':None }
    
    # Using librosa to calculate the features
    features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr)
    features['rolloff'] = librosa.feature.spectral_rolloff(y, sr=sr)
    features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr)
    features['bandwidth'] = librosa.feature.spectral_bandwidth(y, sr=sr)
    features['flatness'] = librosa.feature.spectral_flatness(y)
    features['zcr'] = librosa.feature.zero_crossing_rate(y)
    features['rmse'] = librosa.feature.rms(y)
    features['flux'] = librosa.onset.onset_strength(y=y, sr=sr)
    features['chroma'] = librosa.feature.chroma_stft(y=y, sr=sr)
 
    
    # MFCC treatment
    mfcc = librosa.feature.mfcc(y,n_mfcc=21)
    for idx, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(idx)] = v_mfcc
        
    # Get statistics from the vectors
    def get_moments(descriptors):
        result = {}
        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
        return result
    
    dict_agg_features = get_moments(features)
    dict_agg_features['tempo'] = librosa.beat.tempo(y, sr=sr)[0]
    
    return dict_agg_features

In [7]:
def read_process_songs(src_dir, debug = True):    
    # Empty array of dicts with the processed features from all files
    arr_features = []

    # Read files from the folders
    for x,_ in genres.items():
        folder = src_dir + x
        
        for root, subdirs, files in os.walk(folder):
            for file in files:
                # Read the audio file
                file_name = folder + "/" + file
                signal, sr = librosa.load(file_name, mono=True, duration=30)
                
                # Debug process
                if debug:
                    print("Reading file: {}".format(file_name))
                
                # Append the result to the data structure
                features = get_features(signal, sr)
                features['genre'] = x
                arr_features.append(features)
    return arr_features


In [8]:
%%time

# Get list of dicts with features and convert to dataframe
features = read_process_songs(gtzan_dir, debug=False)

C:\Users\high-\AppData\Local\Temp/ipykernel_2520/1805278335.py:7: FutureWarning: Pass y=[0.00732422 0.01660156 0.00762939 ... 0.18188477 0.1637268  0.15093994] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['centroid'] = librosa.feature.spectral_centroid(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_2520/1805278335.py:8: FutureWarning: Pass y=[0.00732422 0.01660156 0.00762939 ... 0.18188477 0.1637268  0.15093994] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['rolloff'] = librosa.feature.spectral_rolloff(y, sr=sr)
C:\Users\high-\AppData\Local\Temp/ipykernel_2520/1805278335.py:9: FutureWarning: Pass y=[0.00732422 0.01660156 0.00762939 ... 0.18188477 0.1637268  0.15093994] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['contrast'] = librosa.feature.spectral_contrast(y, sr=sr)
C:\Users\high-\AppData\L

C:\Users\high-\AppData\Local\Temp/ipykernel_2520/1805278335.py:11: FutureWarning: Pass y=[-0.01300049 -0.03060913 -0.03607178 ... -0.00515747 -0.00244141
  0.00283813] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['flatness'] = librosa.feature.spectral_flatness(y)
C:\Users\high-\AppData\Local\Temp/ipykernel_2520/1805278335.py:13: FutureWarning: Pass y=[-0.01300049 -0.03060913 -0.03607178 ... -0.00515747 -0.00244141
  0.00283813] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  features['rmse'] = librosa.feature.rms(y)
C:\Users\high-\AppData\Local\Temp/ipykernel_2520/1805278335.py:19: FutureWarning: Pass y=[-0.01300049 -0.03060913 -0.03607178 ... -0.00515747 -0.00244141
  0.00283813] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(y,n_mfcc=21)
C:\Users\high-\AppData\Local\Temp/ipykernel_2520/1805278

KeyboardInterrupt: 

In [ ]:
df_features = pd.DataFrame(features)

In [ ]:
df_features.head()

In [ ]:
df_features.to_csv('gtzan_features.csv', index=False)